<a href="https://colab.research.google.com/github/fuman/nlp/blob/main/createKM_Mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1단계: 환경 설정

In [38]:
!pip install keybert
!pip install nltk
!pip install sklearn
!pip install konlpy  # 한국어 처리를 위한 라이브러리
!pip install kiwipiepy

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# 2단계: 라이브러리 IMPORT

In [47]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt  # 한국어 형태소 분석기
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# 기존 임포트문들도 유지
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import numpy as np

from kiwipiepy import Kiwi
import pandas as pd

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 3단계: 불용어 처리

## 3-1: KIWI 불용어 처리

In [40]:
def get_filtered_text_kwiki(text):

    kiwi = Kiwi()

    filtered_tokens = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        #if len(token) != 1 and pos.startswith('N') or pos.startswith('VV') or pos.startswith('SL'):
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            filtered_tokens.append(token)

    return filtered_tokens

## 3-2: Okt 불용어 처리

In [41]:
def get_filtered_text_Okt(text, lang='ko'):

    if lang == 'en':
        # 영어 텍스트 처리
        tokens = word_tokenize(text.lower())
        pos_tags = pos_tag(tokens)
        filtered_tokens = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('VB')]
    else:
        # 한국어 텍스트 처리
        okt = Okt()
        pos_tags = okt.pos(text)
        #filtered_tokens = [word for word, pos in pos_tags if pos in ['Noun', 'Verb']]
        filtered_tokens = [word for word, pos in pos_tags if pos in ['Noun']]

    return filtered_tokens

# 4단계: 핵심키워드 추출

In [42]:
def get_keywords(text, top_n=5, lang='ko', mode='Okt'):

    #vectorizer = CountVectorizer(ngram_range=(1, 2))  # 2-gram까지 고려
    #kw_model = KeyBERT(vectorizer=vectorizer)

    kw_model = KeyBERT()
    filtered_tokens = []

    if mode == 'kwiki':
        filtered_tokens = get_filtered_text_kwiki(text)
    else:
        filtered_tokens = get_filtered_text_Okt(text, lang)

    filtered_text = ' '.join(filtered_tokens)
    print(filtered_text)

    keywords = kw_model.extract_keywords(filtered_text, keyphrase_ngram_range=(1, 2), top_n=top_n, stop_words='english')
    #keywords = kw_model.extract_keywords(filtered_text, top_n=top_n, stop_words='english')
    print(keywords)
    return [keyword for keyword, _ in keywords]

# 5단계: 문서 Load

In [43]:
def get_file_text():
    # 여러 파일 읽기
    file_list = ['문서1.txt', '문서2.txt', '문서3.txt', '문서4.txt']  # 파일 이름 목록
    df_doc = pd.DataFrame(columns=['파일명', '텍스트'])
    for file_name in file_list:
        with open("/content/drive/MyDrive/01.교육/2024 자연어처리/DAY01/txt/" + file_name, 'r', encoding='utf-8') as file:
            content = file.read()

            #df_doc = df.append({'파일명': file_name, '내용': content}, ignore_index=True)
            new_row = pd.DataFrame({'파일명': [file_name], '텍스트': [content]})
            df_doc = pd.concat([df_doc, new_row], ignore_index=True)

    print(df_doc)

    return df_doc

# 6단계: 실행

In [44]:
# 문서 load
df_doc = get_file_text()

       파일명                                                텍스트
0  문서1.txt  서울 시청역 인근에서 9명 사망이란 대형 교통사고가 났다. 고령 운전자가 역주행을 ...
1  문서2.txt  김포시(시장 김병수)가 4일, 본격적인 AI시대를 대비해 관내 지능형 교통장비를 확...
2  문서3.txt  기술이란 단어는 어떤 문장과 상황에 사용되느냐에 따라 여러 가지로 해석이 가능한 단...
3  문서4.txt  Following the deadly car crash near central Se...


In [45]:
# 리스트 컴프리헨션에서 zip() 함수 사용
#df_doc['키워드_kwiki'] = [get_keywords(text, 5, 'ko', 'kwiki') for text in zip(df_doc['텍스트'])]

# 리스트 컴프리헨션에서 apply()와 람다 함수 사용
df_doc['키워드_kwiki'] = df_doc['텍스트'].apply(lambda x: get_keywords(x, 5, 'ko', 'kwiki')).tolist()

df_doc['키워드_Okt'] = df_doc['텍스트'].apply(lambda x: get_keywords(x, 5, 'ko', 'Okt')).tolist()

df_doc['키워드_Okt_en'] = df_doc['텍스트'].apply(lambda x: get_keywords(x, 5, 'en', 'Okt')).tolist()

df_doc

서울 시청역 인근 사망 교통 사고 고령 운전자 역주행 인도 돌진 보행자 사고 보도 종합 시청역 인근 일방통행 차선 도로 세종대로 역주행 차량 도로 차량 횡단보도 인도 돌진 신호 보행자 사고 남성 남성 남성 남성 남성 여성 남성 부상 가해 운전자 남성 현장 검거 음주 운전 혐의 해당 고령자 급발진 주장 급발진 주장 목격자 급발진 급발진 횡단보도 차량 연합뉴스 인근 차량 블랙박스 영상 급발진 의견 표출 가해자 특성 고령 운전자 비판 X 트위터 실시간 현장 사진 누리 노인 운전자 우리 생명 위협 운전면허 아무 개월 개월 선별 운전 기타 문제 발생 면허 의견 개진
[('인근 일방통행', 0.4873), ('블랙박스 영상', 0.4588), ('일방통행 차선', 0.4579), ('세종대로 역주행', 0.4417), ('운전자 역주행', 0.4225)]
김포시 시장 김병수 본격 AI 시대 대비 관내 지능 교통 장비 확대 지능 교통 체계 ITS 확대 사업 국비 포함 투입 관내 주요 도로 첨단 AI 지능 교통 장비 설치 급속 도시 발전 교통량 증가 교통 혼잡 완화 교통 안전 강화 추진 사업 시작 투입 관내 개소 첨단 신호 제어 교통 정보 수집 제공 첨단 지능 교통 장비 설치 확대 구축 사업 투입 관내 개소 AI 지능 교통 정보 인프라 확대 구축 이후 교통 신호등 온라인 구축 사업 추진 긴급 차량 신호 사업 추진 계획 향후 자율 주행 시대 대비 차세대 지능 교통 체계 C ITS 구축 계획 시민 교통 편의 다각 정책 구상 진석 도시 안전 정보 센터장 김포 대비 다양 교통 정보 정책 계획 추진 시민 편리 교통 환경 노력
[('교통 환경', 0.4168), ('완화 교통', 0.4062), ('교통 신호등', 0.4006), ('김병수 본격', 0.3822), ('안전 강화', 0.3794)]
기술 단어 문장 상황 사용 가지 해석 가능 단어 기술 과학 기술 발전 한자 의미 사용 한자어 상황 Skill Technique Technology 다양 형태 번역 단어 오늘 우리 기술 한자

,파일명,텍스트,키워드_kwiki,키워드_Okt,키워드_Okt_en
0,문서1.txt,서울 시청역 인근에서 9명 사망이란 대형 교통사고가 났다. 고령 운전자가 역주행을 ...,"[인근 일방통행, 블랙박스 영상, 일방통행 차선, 세종대로 역주행, 운전자 역주행]","[블랙박스 영상, 운전자 역주행, 역주행 인도, 남성 명과, 서울 시청역]","[블랙박스 영상이, 시청역 인근에서, 종합하면 시청역, 인근 일방통행인, 이들의 의견도]"
1,문서2.txt,"김포시(시장 김병수)가 4일, 본격적인 AI시대를 대비해 관내 지능형 교통장비를 확...","[교통 환경, 완화 교통, 교통 신호등, 김병수 본격, 안전 강화]","[교통혼잡 완화, 이후 교통신호등, 교통 환경, 완화 교통, 교통신호등]","[완화 교통안전을, 편리한 교통환경을, 지능형교통장비를 설치해, 교통안전을 강화하기..."
2,문서3.txt,기술이란 단어는 어떤 문장과 상황에 사용되느냐에 따라 여러 가지로 해석이 가능한 단...,"[국어사전 정의, 임석진 윤용, 번역 skill, 상황 skill, 용구 생활상]","[국어사전 정의, 문장 상황, 생산 과정, 생활 상의, 건축 용어사전]","[육체노동이나 정신노동의, 국어사전의 정의는, 인간의 생산적, 용구나 생활상의, 조..."
3,문서4.txt,Following the deadly car crash near central Se...,"[accidents younger, korea drivers, seoul drove...",[],"[drivers koreans, korea drivers, seoul drove, ..."


In [46]:
'''
df_doc["키워드_kwiki"] = ""

for index, row in df_doc.iterrows():
    text = row['텍스트']

    #print(index)

    keyword = get_keywords(text, 5, 'ko', 'kwiki')
    #print(keyword)
    #print(type(keyword))

    df_doc.at[index, '키워드_kwiki'] = get_keywords(text, 5, 'ko', 'kwiki')

    #print(index)

    #df_doc.at[index, '키워드_Okt'] = get_keywords(text, 5, 'ko', 'Okt')

    #row['키워드_Okt_en'] = get_keywords(text, 5, 'en', 'Okt')

#print(df_doc)
'''

'\ndf_doc["키워드_kwiki"] = ""\n\nfor index, row in df_doc.iterrows():\n    text = row[\'텍스트\']\n    \n    #print(index)\n\n    keyword = get_keywords(text, 5, \'ko\', \'kwiki\')\n    #print(keyword)\n    #print(type(keyword))\n\n    df_doc.at[index, \'키워드_kwiki\'] = get_keywords(text, 5, \'ko\', \'kwiki\')\n\n    #print(index)\n\n    #df_doc.at[index, \'키워드_Okt\'] = get_keywords(text, 5, \'ko\', \'Okt\')\n\n    #row[\'키워드_Okt_en\'] = get_keywords(text, 5, \'en\', \'Okt\')\n\n#print(df_doc)\n'